In [1]:
#r "nuget:OpenCvSharp4.Windows, 4.2.0.20200208"

using OpenCvSharp;

Installing package OpenCvSharp4.Windows, version 4.2.0.20200208.......

Unhandled exception: input.fsx (2,7)-(2,18) typecheck error The value or constructor 'OpenCvSharp' is not defined.

In [2]:
using System;
using System.Drawing;
using System.IO;
using System.Linq;

In [3]:
var imagesFolderPath = "F:/Palm-vein-ID/small-db"; // path to images
var imagesOutputDirectory = "F:/Palm-vein-ID/small-db/output"; // path to output directory

In [4]:
var binaryROIFolder = "F:/Palm-vein-ID/small-db/output/bin/";

In [5]:
var elementSize = new OpenCvSharp.Size(3, 3);

In [6]:
public class PalmModel {
    public string FileName { get; set; }
    public string Directory { get; set; }
    public OpenCvSharp.Mat SourceImage { get; set; }
    public OpenCvSharp.Mat ThresholdImage { get; set; } // = binary
    
    // get image params
    public int Width { get; set; }
    public int Height { get; set; }
    
    public OpenCvSharp.Mat ROI { get; set; }
    // public OpenCvSharp.Mat FinalThreshold { get; set; }
}

public List<PalmModel> listOfPalms = new List<PalmModel>();

In [7]:
Console.WriteLine($"[{DateTime.Now}] Loading ROI binary dump");

Directory.GetFiles(imagesFolderPath, "*.jpg").ToList().ForEach(x => {
    x = x.Remove(0, x.LastIndexOf('\\') + 1).Replace(".jpg", "");
    using (var fs = new OpenCvSharp.FileStorage($"{binaryROIFolder}{x}.bin", OpenCvSharp.FileStorage.Mode.Read)) {
        listOfPalms.Add(new PalmModel() {
            FileName = fs["FileName"].ReadString(),
            Directory = fs["Directory"].ReadString(),
            SourceImage = fs["SourceImage"].ReadMat(),
            ThresholdImage = fs["ThresholdImage"].ReadMat(),
            Width = fs["Width"].ReadInt(),
            Height = fs["Height"].ReadInt(),
            ROI = fs["ROI"].ReadMat()
        });
    }
});

Console.WriteLine($"[{DateTime.Now}] ROI Dump was loaded.");
Console.WriteLine($"Total palms: {listOfPalms.Count}");
Console.WriteLine (listOfPalms[4].FileName);
Console.WriteLine (listOfPalms[4].Directory);

display(
    span(
        a[href: "#"](b("Sample source image")),
        img[src: "sample/source.jpg", height: 400, width: 400] /* used random image */
    )
);

[05.03.2020 21:19:41] Loading ROI binary dump
[05.03.2020 21:19:43] ROI Dump was loaded.
Total palms: 6
001_l_940_01
F:/Palm-vein-ID/small-db/output\001_l_940_01


Sample source image

In [8]:
display(HTML("<center><b>Preprocessing</b></center><hr />"));

Preprocessing

In [9]:
listOfPalms.ForEach( x => {
    OpenCvSharp.Cv2.MedianBlur (x.ROI, x.ROI, 5);
    // OpenCvSharp.Cv2.CvtColor(x.ROI, x.ROI, ColorConversionCodes.BGR2GRAY);
    OpenCvSharp.Cv2.FastNlMeansDenoising (x.ROI, x.ROI);
    OpenCvSharp.Cv2.CvtColor (x.ROI, x.ROI, OpenCvSharp.ColorConversionCodes.GRAY2BGR);
    OpenCvSharp.Cv2.ImWrite (x.Directory + "\\reduce_noise.jpg", x.ROI);
});

display(
    span(
        a[href: "#"](b("Noise reduced")),
        img[src: "sample/reduce_noise.jpg"] /* used random image */
    )
);

Noise reduced

In [10]:
listOfPalms.ForEach(x =>
{
    var element = OpenCvSharp.Cv2.GetStructuringElement (
        OpenCvSharp.MorphShapes.Cross, 
        elementSize
    ); 
    
    OpenCvSharp.Cv2.MorphologyEx (x.ROI, 
                                  x.ROI, 
                                  OpenCvSharp.MorphTypes.Open, 
                                  element);
    OpenCvSharp.Cv2.CvtColor (x.ROI, 
                              x.ROI, 
                              OpenCvSharp.ColorConversionCodes.BGR2YUV);

    // OpenCvSharp.Cv2.EqualizeHist(x.ROI, x.ROI);
    var RGB = OpenCvSharp.Cv2.Split (x.ROI);

    RGB[0] = RGB[0].EqualizeHist ();
    RGB[1] = RGB[1].EqualizeHist ();
    RGB[2] = RGB[2].EqualizeHist ();

    OpenCvSharp.Cv2.Merge (RGB, x.ROI);
    OpenCvSharp.Cv2.CvtColor (x.ROI, 
                              x.ROI, 
                              OpenCvSharp.ColorConversionCodes.YUV2BGR);

    OpenCvSharp.Cv2.ImWrite (x.Directory + "\\equalized_hist.jpg", x.ROI);
});

display(
    span(
        a[href: "#"](b("Equalized image")),
        img[src: "sample/equalized_hist.jpg"] /* used random image */
    )
);

Equalized image

In [11]:
listOfPalms.ForEach(x =>
{
    OpenCvSharp.Cv2.BitwiseNot (x.ROI, x.ROI);
    OpenCvSharp.Cv2.ImWrite (x.Directory + "\\inverted.jpg", x.ROI);
});

display(
    span(
        a[href: "#"](b("Inverted image")),
        img[src: "sample/inverted.jpg"] /* used random image */
    )
);

Inverted image

In [12]:
listOfPalms.ForEach (x => {
    var element = OpenCvSharp.Cv2.GetStructuringElement (
        OpenCvSharp.MorphShapes.Cross, 
        elementSize
    ); 

    OpenCvSharp.Cv2.CvtColor (x.ROI, 
                              x.ROI, 
                              OpenCvSharp.ColorConversionCodes.BGR2GRAY);
    OpenCvSharp.Cv2.Erode (x.ROI, x.ROI, element);
    OpenCvSharp.Cv2.ImWrite (x.Directory + "\\eroded.jpg", x.ROI);
});

display(
    span(
        a[href: "#"](b("Eroded image")),
        img[src: "sample/eroded.jpg"] /* used random image */
    )
);

Eroded image

In [13]:
listOfPalms.ForEach(x => {
    var skel = new OpenCvSharp.Mat(x.ROI.Size(), OpenCvSharp.MatType.CV_8UC1, new OpenCvSharp.Scalar(0));
    var temp = new OpenCvSharp.Mat();
    var eroded = new OpenCvSharp.Mat();

    var element = OpenCvSharp.Cv2.GetStructuringElement(OpenCvSharp.MorphShapes.Cross, elementSize);

    do {
        OpenCvSharp.Cv2.Erode(x.ROI, eroded, element);
        OpenCvSharp.Cv2.Dilate(eroded, temp, element);
        OpenCvSharp.Cv2.Subtract(x.ROI, temp, temp);
        OpenCvSharp.Cv2.BitwiseOr(skel, temp, skel);
        eroded.CopyTo(x.ROI);

    } while (OpenCvSharp.Cv2.CountNonZero(x.ROI) != 0);

    var thr = skel.Threshold(0, 255, OpenCvSharp.ThresholdTypes.Binary);
    OpenCvSharp.Cv2.ImWrite(x.Directory + "\\thr.jpg", thr);
});

display(
    span(
        a[href: "#"](b("ROI image")),
        img[src: "sample/ROI.jpg"] /* used random image */
    )
);
display(
    span(
        a[href: "#"](b("Final thresholded image")),
        img[src: "sample/thr.jpg"] /* used random image */
    )
);

ROI image

Final thresholded image